# Get Historical Price Data Using Quandl

We'll create a RESTClient to access the Sharadar data from Quandl/Nasdaq Data Link. This will be very similar to [get historical price data from polygon.io](https://analyzingalpha.com/get-historical-price-data-polygon)

Creating our own RESTClient makes our lives easier in the long run. The `__init__` method initializes quandl with various settings. We'll then create a `get_tickers` method and a `get_bars` method for our convenience.

### Links
- [Nasdaq Data Link Quandl API Blog Post](https://analyzingalpha.com/nasdaq-data-link-quandl-api)
- [Nasdaq Data Link Quandl API YouTube Video](https://youtu.be/-MSGRTLc7vc)

## Install Quandl

If you haven't already, activate your virtual environment and install quandl.

In [ ]:
!pip install quandl

### Get Imports

In [1]:
from datetime import date
import quandl
import numpy as np
import pandas as pd
from local_settings import quandl as settings

### Create MyRESTCLient

In [ ]:
class MyRESTClient():
    def __init__(self, auth_key: str=None):
        '''
        Input: dictionary with configuration parameters
        Returns: Configured quandl connection
        '''
        if not ('api_key' in settings.keys() and
            'number_of_retries' in settings.keys() and 
            'max_wait_between_retries' in settings.keys() and
            'retry_backoff_factor' in settings.keys() and
            'retry_status_codes' in settings.keys()):
            raise Exception('Bad quandl config file.')

        # https://github.com/quandl/quandl-python#configuration
        quandl.ApiConfig.api_key = settings['api_key']
        quandl.ApiConfig.NUMBER_OF_RETRIES = settings['number_of_retries']
        quandl.ApiConfig.MAX_WAIT_BEWTEEN_RETRIES = settings['max_wait_between_retries']
        quandl.ApiConfig.RETRY_BACKOFF_FACTOR = settings['retry_backoff_factor']
        quandl.ApiConfig.RETRY_STATUS_CODES = settings['retry_status_codes']

        self._session = quandl

### Create Get Tickers Method

In [ ]:
    def get_tickers(self) -> pd.DataFrame:

        tickers = self._session.get_table('SHARADAR/TICKERS', paginate=True)

        # Filter tickers for equities and funds. Remove instituational tables.
        tickers = tickers[(tickers['table'] == 'SEP') | \
                          (tickers['table'] == 'SFP')]


        # Set NaNs to None and strings to boolean
        tickers.replace({np.nan: None}, inplace=True)

        # Convert isdelated to active
        tickers['active'] = tickers['isdelisted'].apply(lambda x: bool(x == 'N'))

        # Rename and get only fields of interest
        tickers = tickers.rename(columns={'permaticker':'quandl_id',
                                          'siccode':'sic'})

        # Set type of quandl_id to int64
        tickers['quandl_id'] = tickers['quandl_id'].astype(int)

        # Return only columns we want
        cols = ['ticker', 'name', 'active', 'sic',
                'sector', 'industry', 'quandl_id', 'category']
        tickers = tickers[cols]

        # Prevents duplicate data sent by API provider
        tickers = tickers.drop_duplicates(subset='ticker')
        return tickers

### Create Get Bars Method

In [ ]:
    def get_bars(self, market:str='stock', ticker:str=None, from_:date=None, to:date=None) -> pd.DataFrame:

        # Convert np.NaT to None
        from_ = None if pd.isnull(from_) else from_
        to = None if pd.isnull(to) else to

        # Set datea to most recent year if None
        to = to if to else date.today()
        from_ = from_ if from_ else date(2000,1,1)

        tables = ['SHARADAR/SEP', 'SHARADAR/SFP']
        
        for table in tables:
            df = self._session.get_table(table,
                                         ticker=ticker,
                                         date={'gte':from_,'lte':to},
                                         paginate=True)

            if not df.empty:
                df['date'] = pd.to_datetime(df['date'])
                df = df.sort_values(by='date')
                df = df[['date', 'open','high','low','close','volume']]
                return df

        return None

## The Class

In [2]:
class MyRESTClient():
    def __init__(self, auth_key: str=None):
        '''
        Input: dictionary with configuration parameters
        Returns: Configured quandl connection
        '''
        if not ('api_key' in settings.keys() and
            'number_of_retries' in settings.keys() and 
            'max_wait_between_retries' in settings.keys() and
            'retry_backoff_factor' in settings.keys() and
            'retry_status_codes' in settings.keys()):
            raise Exception('Bad quandl config file.')

        # https://github.com/quandl/quandl-python#configuration
        quandl.ApiConfig.api_key = settings['api_key']
        quandl.ApiConfig.NUMBER_OF_RETRIES = settings['number_of_retries']
        quandl.ApiConfig.MAX_WAIT_BEWTEEN_RETRIES = settings['max_wait_between_retries']
        quandl.ApiConfig.RETRY_BACKOFF_FACTOR = settings['retry_backoff_factor']
        quandl.ApiConfig.RETRY_STATUS_CODES = settings['retry_status_codes']

        self._session = quandl
        
    def get_tickers(self) -> pd.DataFrame:
        tickers = self._session.get_table('SHARADAR/TICKERS', paginate=True)

        # Filter tickers for equities and funds. Remove instituational tables.
        tickers = tickers[(tickers['table'] == 'SEP') | \
                          (tickers['table'] == 'SFP')]

        # Set NaNs to None and strings to boolean
        tickers.replace({np.nan: None}, inplace=True)

        # Convert isdelated to active
        tickers['active'] = tickers['isdelisted'].apply(lambda x: bool(x == 'N'))

        # Rename and get only fields of interest
        tickers = tickers.rename(columns={'permaticker':'quandl_id',
                                          'siccode':'sic'})
        
        # Set type of quandl_id to int64
        tickers['quandl_id'] = tickers['quandl_id'].astype(int)

        # Return only columns we want
        cols = ['ticker', 'name', 'active', 'sic',
                'sector', 'industry', 'quandl_id', 'category']
        tickers = tickers[cols]

        # Prevents duplicate data sent by API provider
        tickers = tickers.drop_duplicates(subset='ticker')
        return tickers
    
    def get_bars(self, market:str='stock', ticker:str=None,
                  from_:date=None, to:date=None) -> pd.DataFrame:

        # Convert np.NaT to None
        from_ = None if pd.isnull(from_) else from_
        to = None if pd.isnull(to) else to

        # Set datea to most recent year if None
        to = to if to else date.today()
        from_ = from_ if from_ else date(2000,1,1)

        tables = ['SHARADAR/SEP', 'SHARADAR/SFP']
        
        for table in tables:
            df = self._session.get_table(table,
                                         ticker=ticker,
                                         date={'gte':from_,'lte':to},
                                         paginate=True)

            if not df.empty:
                df['date'] = pd.to_datetime(df['date'])
                df = df.sort_values(by='date')
                df = df[['date', 'open','high','low','close','volume']]
                return df

        return None

## Get Apple's Prices

In [3]:
client = MyRESTClient()
client.get_bars(ticker='AAPL')

,date,open,high,low,close,volume
None,,,,,,
5489,2000-01-03,0.936,1.004,0.908,1.000,535796800.0
5488,2000-01-04,0.967,0.988,0.903,0.915,512377600.0
5487,2000-01-05,0.926,0.987,0.920,0.928,778321600.0
5486,2000-01-06,0.948,0.955,0.848,0.848,767972800.0
5485,2000-01-07,0.862,0.902,0.853,0.888,460734400.0
...,...,...,...,...,...,...
4,2021-10-20,148.700,149.754,148.120,149.260,58085532.0
3,2021-10-21,148.810,149.640,147.870,149.480,61247633.0
2,2021-10-22,149.690,150.180,148.640,148.690,58703843.0
